In [1]:
# Training convolutional networks

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
# Load the pickle file

pickle_file = '/Users/rgparekh/Documents/Personal/Rajesh/Data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
# Re-shape the data sets to be TensorFlow friendly

image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
# Define accuracy

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [10]:
# CNN architecture with max-pooling

# Define the CNN architecture

batch_size = 256
patch_size = 5
depth1 = 16
depth2 = 32
num_hidden = 256
num_channels = 1

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  keep_prob = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 14 * image_size // 14 * depth2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob):
    # data is the input data
    # layer1_weights define the filter parameter [filter_height, filter_width, input_channels, output_channels]
    # [1,2,2,1] defines the strides parameter - 1D tensor of length 4 (batch, height, width, channels)
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    
    # conv1 returns a tensor of size [batch_size, image_size/2, image_size/2, depth1]
    
    hidden1 = tf.nn.relu(conv1 + layer1_biases)
    
    # hidden1 returns a tensor of size [batch_size, image_size/2, image_size/2, depth1]
    
    pool1 = tf.nn.max_pool(value=hidden1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    # pool1 returns a tensor of size [batch_size, image_size/4, image_size/4, depth1] (dim reduction since strides = 2)    
    
    #print("Conv1 ", conv1.get_shape().as_list())
    #print("Hidden1 ", hidden1.get_shape().as_list())
    #print("Pool1 ", pool1.get_shape().as_list())
    
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 2, 2, 1], padding='SAME')
    
   # conv2 returns a tensor of size [batch_size, image_size/4/2, image_size/4/2, depth2]

    hidden2 = tf.nn.relu(conv2 + layer2_biases)
    
  # hidden2 returns a tensor of size [batch_size, image_size/8, image_size/8, depth2]

    pool2 = tf.nn.max_pool(value=hidden2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
  # pool2 returns a tensor of size [batch_size, image_size/14, image_size/14, depth2]
    
    #print("Conv2 ", conv2.get_shape().as_list())    
    #print("Hidden2 ", hidden2.get_shape().as_list())    
    #print("Pool2 ", pool2.get_shape().as_list())
    
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    dropout = tf.nn.dropout(hidden3, keep_prob)  # DROP-OUT here
    
    return tf.matmul(dropout, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, 0.5)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.AdamOptimizer(0.005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))


In [11]:
# Train model and compute test set accuracy

num_steps = 2001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.842926
Minibatch accuracy: 11.3%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.012220
Minibatch accuracy: 71.5%
Validation accuracy: 73.6%
Minibatch loss at step 100: 0.721570
Minibatch accuracy: 80.5%
Validation accuracy: 80.6%
Minibatch loss at step 150: 0.592584
Minibatch accuracy: 82.4%
Validation accuracy: 83.8%
Minibatch loss at step 200: 0.558420
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 250: 0.499614
Minibatch accuracy: 84.8%
Validation accuracy: 85.1%
Minibatch loss at step 300: 0.557684
Minibatch accuracy: 82.4%
Validation accuracy: 85.3%
Minibatch loss at step 350: 0.628219
Minibatch accuracy: 80.5%
Validation accuracy: 85.9%
Minibatch loss at step 400: 0.416102
Minibatch accuracy: 87.9%
Validation accuracy: 86.4%
Minibatch loss at step 450: 0.530180
Minibatch accuracy: 85.9%
Validation accuracy: 86.6%
Minibatch loss at step 500: 0.578073
Minibatch accuracy: 81.2%
Validation accuracy: 86.7%
M